In [1]:
import os

In [2]:
%pwd

'd:\\github\\Github\\Wine-Quality-End-to-End-ML-Project\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\github\\Github\\Wine-Quality-End-to-End-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def _map_quality(self, score):
        """Private method to map wine quality to Low/Medium/High."""
        if score <= 5:
            return 'Low'
        else:
            return 'High'

    def train_test_spliting(self):
        """Method to load data and split into train/test sets."""
        logger.info("Loading data")
        # Load data
        data = pd.read_csv(self.config.data_path)

        # Map the quality score to categorical labels
        data['quality_label'] = data['quality'].apply(self._map_quality)

        # Drop the original quality column
        data.drop(columns='quality', inplace=True)

        # Split the data into training and test sets
        train, test = train_test_split(data, test_size=0.2, random_state=42)

        # Save the transformed datasets
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Mapped wine quality and split data into training and test sets")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print(train.shape)
        print(test.shape)


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-04-15 11:30:48,429: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-15 11:30:48,432: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-15 11:30:48,436: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-15 11:30:48,438: INFO: common: created directory at: artifacts]
[2025-04-15 11:30:48,439: INFO: common: created directory at: artifacts/data_transformation]
[2025-04-15 11:30:48,440: INFO: 1603897144: Loading data]
[2025-04-15 11:30:48,572: INFO: 1603897144: Mapped wine quality and split data into training and test sets]
[2025-04-15 11:30:48,574: INFO: 1603897144: Train shape: (914, 13)]
[2025-04-15 11:30:48,576: INFO: 1603897144: Test shape: (229, 13)]
(914, 13)
(229, 13)
